import urllib.request

url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
week_nums = [190504, 190511, 190518, 190525, 190601, 190608, 190615, 190622, 190629, 190706, 190713, 190720, 190727]

for week_num in week_nums:
    urllib.request.urlretrieve(f"http://web.mta.info/developers/data/nyct/turnstile/turnstile_{week_num}.txt",
                               f"data/turnstile_{week_num}.txt")

In [12]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine("sqlite:///mta.db")
all_data = pd.read_sql('SELECT * FROM mta_data;', engine)

In [20]:
df = all_data

# Need to change columns to row-index 0
#df.rename(columns = df.loc[0], inplace =True)

# Remove row 0
#df = df.drop([0], inplace=True)

# Need to convert EXITS column field11 into numeric type
df['EXITS'] = df['EXITS'].apply(pd.to_numeric)

#Drop dups
#.df.drop_duplicates()


In [33]:
df.head(4)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,00:00:00,REGULAR,0007035249,2384833
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,04:00:00,REGULAR,0007035269,2384840
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,08:00:00,REGULAR,0007035292,2384875
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,12:00:00,REGULAR,0007035392,2384951


In [26]:
# Need to find the max(EXITS) for each (SCP)
max_exit = df.groupby(['C/A','SCP'])[['EXITS']].max().head(5)
max_exit

EXITS
C/A  SCP              
A002 02-00-00  2418903
     02-00-01  1425872
     02-03-00  4751266
     02-03-01  1824315
     02-03-02  8432345

In [27]:
# Need to find the min(EXITS) for each (SCP)
min_exit = df.groupby(['C/A','SCP'])[['EXITS']].min().head(5)
min_exit

EXITS
C/A  SCP              
A002 02-00-00  2384833
     02-00-01  1405759
     02-03-00  4603019
     02-03-01  1670332
     02-03-02  8316344

In [42]:
#Merging new exits data into one df, on matching SCP column
total_scp_exit = pd.merge(max_exit, min_exit, on=['C/A','SCP'])
total_scp_exit

EXITS_x  EXITS_y
C/A  SCP                       
A002 02-00-00  2418903  2384833
     02-00-01  1425872  1405759
     02-03-00  4751266  4603019
     02-03-01  1824315  1670332
     02-03-02  8432345  8316344

In [32]:
# total_scp_exits = max-min for a turnstile SCP
total_scp_exit['total'] = total_scp_exit['EXITS_x']-total_scp_exit['EXITS_y']
total_scp_exit

,EXITS_x,EXITS_y,total
SCP,,,
02-00-00,2418903,2384833,34070
02-00-01,1425872,1405759,20113
02-03-00,4751266,4603019,148247
02-03-01,1824315,1670332,153983
02-03-02,8432345,8316344,116001


In [ ]:
#Count unique number of dates

In [ ]:
#Make new column daily_avg_exits
total_scp_exit['daily_avg_exits'] = total_scp_exit['total']

In [ ]:
#Need to JOIN and GROUP BY C/A on SCP

In [ ]:
# Group by (C/A) combine SUM(total_scp_exits) for each (C/A) = total_station_exits